In [1]:
import hoomd
import numpy

# Setting initial conditions in python

You can set initial conditions for HOOMD using arbitrary python code via a snapshot. Use this to generate random initial conditions within a job script, provide customizable parameters for the initial conditions, or read from a custom file format.

## Make Snapshot

The first step is to make a snapshot to hold the particles. `N` is the number of particles to create, `box` is the simulation box, and `particle_types` is a list of the particle type names. This only demonstrates the syntax you can use.

In [2]:
hoomd.context.initialize("");
snap = hoomd.data.make_snapshot(N=4, box=hoomd.data.boxdim(L=10), particle_types=['A', 'B']);

HOOMD-blue v2.1.5 CUDA (7.5) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 03/09/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): This system is not compute exclusive, using local rank to select GPUs
notice(2): Unable to identify node local rank information
notice(2): Using global rank to select GPUs
HOOMD-blue is running on the following GPU(s):
 [0]            Tesla K20c  13 SM_3.5 @ 0.706 GHz, 5060 MiB DRAM


The snapshot object contains [numpy](http://www.numpy.org/) arrays with all of the particle properties.

In [3]:
print(snap.particles.position)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [4]:
print(snap.particles.diameter)

[ 1.  1.  1.  1.]


In [5]:
print(snap.particles.velocity)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


As you can see, all particles are initialized with default properties.

## Populate snapshot

The next step is to populate the snapshot with your desired initial condition. This can be created by any code, or assigned by hand. Make sure to assign positions for all particles.

In [6]:
snap.particles.position[0] = [1,2,3];
snap.particles.position[1] = [-1,-2,-3];
snap.particles.position[2] = [3,2,1];
snap.particles.position[3] = [-3,-2,-1];

You must assign elements of the arrays in the snapshot (particles.position[...] = ...), not the arrays themselves (particles.position = array). You can copy an existing numpy array with the [:] syntax.

In [7]:
numpy.random.seed(10);
my_velocity = numpy.random.random((4,3)) * 2 - 1;
snap.particles.velocity[:] = my_velocity[:];
print(snap.particles.velocity)

[[ 0.54264128 -0.95849609  0.26729646]
 [ 0.49760777 -0.00298598 -0.55040669]
 [-0.60387427  0.52106142 -0.66177833]
 [-0.82332039  0.37071964  0.90678668]]


We assigned `particle_types=['A', 'B']` above. This makes the type id of 0 identify particle type 'A', and type id 1 identify particle type 'B'. The type id is just the index of the name in the list. Make the first three particles 'B' and the last one 'A':

In [8]:
snap.particles.typeid[:] = [1,1,1,0];

## Initialize from snapshot

The snapshot object `snap` stores a self-contained version of the system configuration. It is not connected with the rest of HOOMD. You can store as many snapshot objects as you can fit in memory, all with different properties. Use `hoomd.init.read_snapshot` to initialize a HOOMD simulation from a snapshot.

In [9]:
hoomd.init.read_snapshot(snap);

notice(2): Group "all" created containing 4 particles


Now HOOMD is ready for pair forces, integrators, and other commands to configure the simulation run.